In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier # Uncomment if CatBoost is installed
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score,
    matthews_corrcoef, precision_score, recall_score,
    average_precision_score, confusion_matrix
)
import numpy as np
import re
import sys
import warnings

# Import imblearn techniques
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import Pipeline # Although we apply techniques manually here, Pipeline is useful

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning) # imblearn might raise these

# --- Data Loading and Filtering ---
# Load all necessary data files
try:
    rna_data = pd.read_csv('D:\\extensive analysis data\\rna_samples_with_gene_names.csv')
    mirna_data = pd.read_csv('D:\\extensive analysis data\\miRNA_samples.csv')
    methylation_data = pd.read_csv("D:\\extensive analysis data\\methylation_samples.csv")
    cnv_data = pd.read_csv("D:\\extensive analysis data\\cnv_samples.csv")
    protein_data = pd.read_csv("D:\\extensive analysis data\\protein_samples.csv")
    clin_data = pd.read_csv('D:\\extensive analysis data\\clin_common.csv')
    # Load the pooled data file containing the features to keep
    pooled_data = pd.read_csv('D:\\Downloads\\Top Features - All Pooled.csv')
    print("Data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading data file: {e}")
    print("Please check the file paths in the code.")
    sys.exit(1) # Exit the script if files are not found

# Filter omics data based on features listed in pooled_data
print("\nFiltering omics data based on pooled features...")

# RNA filtering
rna_features = pooled_data['Gene'].dropna().unique().tolist()
# Ensure 'Unnamed: 0' (sample ID column) is always included
rna_data_filtered = rna_data[['Unnamed: 0'] + [col for col in rna_features if col in rna_data.columns]].copy()

# MiRNA filtering
mirna_features = pooled_data['miRNA'].dropna().unique().tolist()
mirna_data_filtered = mirna_data[['Unnamed: 0'] + [col for col in mirna_features if col in mirna_data.columns]].copy()

# CNV filtering
cnv_features = pooled_data['CNV'].dropna().unique().tolist()
# CNV data had a header row that wasn't removed before, let's drop the first row if it exists and looks like a header
if cnv_data.iloc[0].astype(str).str.contains('gene_id').any(): # Check if the first row contains 'gene_id'
     print("Dropping header row from CNV data.")
     cnv_data = cnv_data.iloc[1:].copy()
else:
    cnv_data = cnv_data.copy() # Just copy if no header row detected

# Convert columns to numeric after dropping header, coerce errors
for col in cnv_data.columns:
    if col != 'Unnamed: 0': # Avoid converting sample ID
        cnv_data[col] = pd.to_numeric(cnv_data[col], errors='coerce')

# Filter CNV data, exclude 'gene_id' if it exists as a column name in features
cnv_data_filtered = cnv_data[['Unnamed: 0'] + [col for col in cnv_features if col in cnv_data.columns and col != 'gene_id']].copy()


# Methylation filtering
methylation_features = pooled_data['methylation'].dropna().unique().tolist()
methylation_data_filtered = methylation_data[['Unnamed: 0'] + [col for col in methylation_features if col in methylation_data.columns]].copy()

# Protein filtering
protein_features = pooled_data['Protein'].dropna().unique().tolist()
protein_data_filtered = protein_data[['Unnamed: 0'] + [col for col in protein_features if col in protein_data.columns]].copy()


print("Data filtering complete.")
print(f"RNA filtered shape: {rna_data_filtered.shape}")
print(f"MiRNA filtered shape: {mirna_data_filtered.shape}")
print(f"CNV filtered shape: {cnv_data_filtered.shape}")
print(f"Methylation filtered shape: {methylation_data_filtered.shape}")
print(f"Protein filtered shape: {protein_data_filtered.shape}")


# --- Helper Functions ---

def clean_sample_ids(sample_id_list):
    """Cleans sample IDs to a base format (e.g., TCGA-XX-XXXX-XX)."""
    cleaned_ids = []
    for sid in sample_id_list:
        sid_str = str(sid)
        # Use the pattern that matches the sample ID format
        match = re.match(r'TCGA-\w{2}-\w{4}-\w{2}', sid_str)
        if match:
            cleaned_ids.append(match.group(0))
        else:
            # If it doesn't match the standard pattern, keep the original or handle as error
            # print(f"Warning: Sample ID '{sid_str}' did not match expected pattern. Keeping original.")
            cleaned_ids.append(sid_str) # Keep original if no match
    return cleaned_ids

def merge_omics_clinical(omics_df, clinical_df, omics_name):
    """Merges omics and clinical data, handling sample ID cleaning."""
    print(f"\n--- Processing {omics_name} Data ---")
    print(f"Merging {omics_name} data...")
    # Ensure 'Unnamed: 0' exists in omics_df and 'sample_id.1' in clinical_df
    if 'Unnamed: 0' not in omics_df.columns:
        print(f"Error: '{omics_name}' dataframe is missing the 'Unnamed: 0' column for merging. Skipping.")
        return None
    if 'sample_id.1' not in clinical_df.columns:
        print(f"Error: Clinical dataframe is missing the 'sample_id.1' column for merging. Skipping {omics_name}.")
        return None

    # Clean omics sample IDs
    omics_df['cleaned_sample_id'] = clean_sample_ids(omics_df['Unnamed: 0'])
    # Clean clinical sample IDs (use 'sample_id.1' as it seems to be the correct identifier)
    clinical_df['cleaned_sample_id'] = clean_sample_ids(clinical_df['sample_id.1'])

    # Merge
    # Use an inner merge to keep only samples present in both datasets
    merged_df = pd.merge(omics_df, clinical_df[['cleaned_sample_id', 'stage_classification']],
                         on='cleaned_sample_id', how='inner')

    # Drop only the original omics ID column, keep 'cleaned_sample_id'
    merged_df = merged_df.drop(['Unnamed: 0'], axis=1)

    print(f"Merge complete. Shape: {merged_df.shape}")
    return merged_df

def handle_missing_values(df, strategy='median'):
    """Handles missing values using the specified strategy."""
    # print(f"Handling missing values using strategy: {strategy}...")
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

    if strategy == 'median':
        for col in numerical_cols:
            df[col] = df[col].fillna(df[col].median())
    elif strategy == 'mean':
        for col in numerical_cols:
            df[col] = df[col].fillna(df[col].mean())
    elif strategy == 'drop_rows':
        df = df.dropna()
    else:
        raise ValueError("Unsupported imputation strategy")

    # print(f"Missing value handling complete. Shape: {df.shape}")
    return df

# Use the evaluate_binary_model function provided by the user
def evaluate_binary_model(y_true_binary, y_pred_binary, y_prob_binary=None, model_name="", omics_name="", technique_name=""):
    """Evaluates a trained binary model and returns a dictionary of metrics."""
    metrics = {}

    # Calculate metrics
    metrics["Accuracy"] = accuracy_score(y_true_binary, y_pred_binary)
    metrics["Balanced Accuracy"] = balanced_accuracy_score(y_true_binary, y_pred_binary)
    metrics["MCC"] = matthews_corrcoef(y_true_binary, y_pred_binary)

    # Precision, Recall, F1 are for the positive class (1) in binary classification
    # Ensure zero_division is handled to avoid warnings/errors on no positive predictions
    precision = precision_score(y_true_binary, y_pred_binary, pos_label=1, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, pos_label=1, zero_division=0) # Sensitivity
    f1 = f1_score(y_true_binary, y_pred_binary, pos_label=1, zero_division=0)

    metrics["Precision"] = precision
    metrics["Recall (Sensitivity)"] = recall
    metrics["F1-Score"] = f1


    # Specificity (Recall of the negative class)
    unique_test_binary_classes = np.unique(y_true_binary)
    if len(unique_test_binary_classes) < 2:
        specificity = np.nan # Cannot calculate specificity if only one class in test set
    else:
        cm = confusion_matrix(y_true_binary, y_pred_binary)
        # Ensure cm has the expected shape (2x2) even if predictions are all one class
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        else:
            # Handle cases where confusion_matrix might return a different shape
            if 0 not in unique_test_binary_classes: # Only positive class (1) in test set
                specificity = 0.0 # No true negatives possible
            elif 1 not in unique_test_binary_classes: # Only negative class (0) in test set
                specificity = 1.0 # All negatives correctly classified as negative
            else:
                # Fallback for unexpected shapes
                print(f"Warning: Unexpected confusion matrix shape {cm.shape} for evaluation ({omics_name} {model_name} {technique_name}). Specificity set to NaN.")
                specificity = np.nan

    metrics["Specificity"] = specificity

    # G-Mean
    if np.isnan(recall) or np.isnan(specificity):
        metrics["G-Mean"] = np.nan
    else:
        metrics["G-Mean"] = np.sqrt(recall * specificity)


    # AUC metrics require probability predictions
    metrics["ROC-AUC"] = np.nan
    metrics["PR-AUC"] = np.nan

    if y_prob_binary is not None:
        try:
            if len(np.unique(y_true_binary)) > 1: # Need both classes in test set for AUC
                 # Check if y_prob contains finite values before calculating AUC
                if np.isfinite(y_prob_binary).all():
                    metrics["ROC-AUC"] = roc_auc_score(y_true_binary, y_prob_binary)
                    metrics["PR-AUC"] = average_precision_score(y_true_binary, y_prob_binary)
                else:
                     print(f"Warning: Probability predictions contain non-finite values for evaluation ({omics_name} {model_name} {technique_name}). Cannot calculate AUC metrics.")
                     metrics["ROC-AUC"] = np.nan
                     metrics["PR-AUC"] = np.nan
            else:
                # Warning already printed if test set has only one class
                pass

        except ValueError as e:
            print(f"Error calculating AUC for evaluation ({omics_name} {model_name} {technique_name}): {e}")
            metrics["ROC-AUC"] = np.nan
            metrics["PR-AUC"] = np.nan
        except Exception as e:
            print(f"An unexpected error occurred during AUC calculation for evaluation ({omics_name} {model_name} {technique_name}): {e}")
            metrics["ROC-AUC"] = np.nan
            metrics["PR-AUC"] = np.nan

    return metrics


def get_feature_importance(model, feature_names, top_n=50):
    """Extracts and returns top N feature importances or coefficients."""
    importance_dict = {}
    model_type = type(model).__name__

    try:
        if hasattr(model, 'feature_importances_'):
            # Tree-based models
            importances = model.feature_importances_
            sorted_indices = np.argsort(importances)[::-1]
            top_features = [(feature_names[i], importances[i]) for i in sorted_indices[:top_n]]
            importance_dict = dict(top_features)
            # print(f"Extracted top {top_n} feature importances for {model_type}.")

        elif hasattr(model, 'coef_'):
            # Linear models (Logistic Regression, SVC with linear kernel)
            # For binary classification, coef_ is shape (1, n_features) or (n_features,)
            # For multi-class (OvR or OvO), it's shape (n_classes, n_features)
            # We are doing binary 'Early vs Not Early', so expect (1, n_features) or (n_features,)
            if model.coef_.ndim > 1:
                 # If it's multi-class coef (from OvR or OvO), take the coefficients for the positive class (index 1)
                # This might need adjustment based on how the model handles binary internally, but for LR it's usually 1 row.
                # Let's assume it's a single row for the binary case or take the first row if it's (1, n_features)
                 coefs = model.coef_[0] if model.coef_.shape[0] == 1 else model.coef_[0] # Take the first row if shape is (1, n_features)
                 if model.coef_.shape[0] > 1:
                      print(f"Warning: Model {model_type} has multi-dimensional coef_ ({model.coef_.shape}). Taking the first row for feature importance.")
            else:
                coefs = model.coef_ # Shape (n_features,)

            # Use absolute values for importance for ranking, but store original coefficient
            abs_coefs = np.abs(coefs)
            sorted_indices = np.argsort(abs_coefs)[::-1]
            top_features = [(feature_names[i], coefs[i]) for i in sorted_indices[:top_n]] # Store original coefficient
            importance_dict = dict(top_features)
            # print(f"Extracted top {top_n} coefficients (as importance) for {model_type}.")

        else:
            # Models that don't support direct feature importance/coefficients
            # print(f"Model {model_type} does not support direct feature importance extraction.")
            pass # Return empty dictionary

    except Exception as e:
        print(f"Error extracting feature importance for {model_type}: {e}")
        importance_dict = {} # Return empty dictionary on error

    return importance_dict


# --- Ensemble Configuration ---
ensemble_config = {
    "Protein": {"model": ExtraTreesClassifier(random_state=42), "technique": BorderlineSMOTE(random_state=42), "omics_df": protein_data_filtered},
    "Methylation": {"model": RandomForestClassifier(random_state=42), "technique": ADASYN(random_state=42), "omics_df": methylation_data_filtered},
    "MiRNA": {"model": RandomForestClassifier(random_state=42), "technique": ADASYN(random_state=42), "omics_df": mirna_data_filtered},
    "CNV": {"model": RandomForestClassifier(random_state=42), "technique": "Class Weighting (Balanced)", "omics_df": cnv_data_filtered},
    "RNA": {"model": RandomForestClassifier(random_state=42), "technique": BorderlineSMOTE(random_state=42), "omics_df": rna_data_filtered},
}

# --- Main Ensemble Building and Evaluation Logic ---

trained_base_models = {}
all_feature_importances = []
test_probabilities = {} # To store probabilities for soft voting

# First, merge clinical data with each omics dataset and get the binary target
merged_omics_data = {}
for omics_name, config in ensemble_config.items():
     omics_df = config["omics_df"].copy()
     merged_df = merge_omics_clinical(omics_df, clin_data.copy(), omics_name)
     if merged_df is not None:
          merged_df['binary_target'] = (merged_df['stage_classification'] == 'Early Stage').astype(int)
          merged_omics_data[omics_name] = merged_df
     else:
          print(f"Skipping {omics_name} for ensemble due to merge failure.")


# Check if we have any data to proceed
if not merged_omics_data:
    print("No omics data successfully merged. Cannot build ensemble. Exiting.")
    sys.exit()

# Get a common set of sample IDs present in all merged datasets
common_sample_ids = None
for omics_name, merged_df in merged_omics_data.items():
    # Ensure the index is reset or use a unique ID column if available after merge
    # We modified merge_omics_clinical to keep 'cleaned_sample_id'
    if 'cleaned_sample_id' in merged_df.columns:
        merged_df_indexed = merged_df.set_index('cleaned_sample_id')
    else:
        print(f"Error: 'cleaned_sample_id' not found in {omics_name} merged data. Cannot proceed with consistent sample splitting. Exiting.")
        sys.exit()

    current_ids = merged_df_indexed.index.tolist()
    if common_sample_ids is None:
        common_sample_ids = set(current_ids)
    else:
        common_sample_ids.intersection_update(current_ids)

    # Update the entry in merged_omics_data with the indexed dataframe for consistent access
    merged_omics_data[omics_name] = merged_df_indexed


common_sample_ids = list(common_sample_ids)

if not common_sample_ids:
     print("No common samples found across all merged omics datasets. Cannot build ensemble. Exiting.")
     sys.exit()

print(f"\nFound {len(common_sample_ids)} common samples across all omics.")

# Use the common sample IDs to create a consistent split across all omics
# We'll split the indices/sample IDs first
train_indices, test_indices = train_test_split(
    common_sample_ids,
    test_size=0.25,
    random_state=42,
    # Stratify based on the binary target from one of the omics (assuming target distribution is similar)
    # Use the target from the first successfully merged and indexed omics
    stratify=merged_omics_data[list(merged_omics_data.keys())[0]].loc[common_sample_ids, 'binary_target']
)

print(f"\nData split into {len(train_indices)} training samples and {len(test_indices)} testing samples.")

# Store the true binary target for the test set (consistent across all omics)
y_test_binary = merged_omics_data[list(merged_omics_data.keys())[0]].loc[test_indices, 'binary_target']
print("Binary Test set class distribution:", pd.Series(y_test_binary).value_counts().sort_index())

if len(np.unique(y_test_binary)) < 2:
    print("Warning: Binary test set contains fewer than 2 classes. AUC metrics for ensemble will be NaN.")


print("\nTraining individual base models and collecting test probabilities...")

# Train each base model and collect test probabilities
for omics_name, config in ensemble_config.items():
    print(f"\n--- Training {omics_name} Base Model ---")
    merged_df_indexed = merged_omics_data.get(omics_name) # Get the pre-merged and indexed dataframe

    if merged_df_indexed is None:
        print(f"Skipping {omics_name} as merged data was not available.")
        continue

    # Select data for the current omics using the common train/test indices
    X_omics = merged_df_indexed.drop(['stage_classification', 'binary_target'], axis=1)
    y_omics_binary = merged_df_indexed['binary_target'] # Should be same as y_binary from the first omics

    X_train_omics = X_omics.loc[train_indices].copy()
    X_test_omics = X_omics.loc[test_indices].copy()
    y_train_omics_binary = y_omics_binary.loc[train_indices].copy() # Use omics-specific target for training

    # Handle Missing Values (Fit on training data for this omics)
    X_train_omics = handle_missing_values(X_train_omics, strategy='median')
    X_test_omics = handle_missing_values(X_test_omics, strategy='median')
    print(f"Missing values handled for {omics_name} using median imputation.")

    # Preprocess Features (Scaling - Fit on training data for this omics)
    scaler_omics = StandardScaler()
    X_train_omics_scaled = scaler_omics.fit_transform(X_train_omics)
    X_test_omics_scaled = scaler_omics.transform(X_test_omics)
    feature_names_omics = X_train_omics.columns.tolist() # Get feature names for this omics

    # Get model and technique from config
    model_instance_original = config["model"]
    technique = config["technique"]
    model_name = type(model_instance_original).__name__
    technique_name = technique if isinstance(technique, str) else type(technique).__name__

    current_model = None
    X_train_res = X_train_omics_scaled.copy()
    y_train_res = y_train_omics_binary.copy()

    try:
        # --- Apply Technique ---
        if technique_name == "None":
            # No sampling, use base model with default/scale_pos_weight
            current_model = model_instance_original.__class__(**model_instance_original.get_params())
            # Apply scale_pos_weight for XGBoost/LightGBM if no sampling
            train_binary_counts_omics = pd.Series(y_train_omics_binary).value_counts().sort_index()
            scale_pos_weight_value_omics = 1.0
            if 0 in train_binary_counts_omics.index and 1 in train_binary_counts_omics.index and train_binary_counts_omics[1] != 0:
                scale_pos_weight_value_omics = train_binary_counts_omics[0] / train_binary_counts_omics[1]
                print(f"  Calculated scale_pos_weight for {omics_name}: {scale_pos_weight_value_omics:.4f}")
            else:
                 print(f"  Warning: Could not calculate scale_pos_weight for {omics_name} (minority class count is zero). Setting to 1.0.")

            if model_name in ["XGBClassifier", "LGBMClassifier"]:
                 current_model.set_params(scale_pos_weight=scale_pos_weight_value_omics)
            # Ensure class_weight is not set if it's not the dedicated technique
            if model_name in ["LogisticRegression", "DecisionTreeClassifier", "SVC", "RandomForestClassifier", "ExtraTreesClassifier"] and 'class_weight' in current_model.get_params():
                 current_model.set_params(class_weight=None)


            print(f"  Training {model_name} for {omics_name} with None (no sampling)...")
            current_model.fit(X_train_res, y_train_res)

        elif technique_name == "Class Weighting (Balanced)":
            # Apply class_weight='balanced' for models that support it
            if model_name in ["LogisticRegression", "DecisionTreeClassifier", "SVC", "RandomForestClassifier", "ExtraTreesClassifier"]:
                current_model = model_instance_original.__class__(**model_instance_original.get_params())
                current_model.set_params(class_weight='balanced')
                print(f"  Training {model_name} for {omics_name} with Class Weighting (Balanced)...")
                current_model.fit(X_train_res, y_train_res)
            else:
                print(f"  Skipping Class Weighting for {model_name} ({omics_name}) as it's not directly supported via class_weight='balanced'.")
                continue # Skip this omics/model combination if class weighting is requested but not supported

        else:
            # Apply sampling techniques from imblearn
            sampler = technique # technique is already the sampler instance

            # Check if sampling is possible (requires min samples for some techniques)
            min_samples_needed = 2 # Default for most samplers
            if technique_name in ["BorderlineSMOTE", "ADASYN", "SMOTETomek", "SMOTEENN"]:
                min_samples_needed = 6 # BorderlineSMOTE, ADASYN defaults k_neighbors=5, need k_neighbors+1 samples

            train_binary_counts_omics = pd.Series(y_train_omics_binary).value_counts().sort_index()
            if train_binary_counts_omics.min() < min_samples_needed and technique_name not in ["RandomOverSampler", "RandomUnderSampler"]:
                print(f"  Skipping {technique_name} for {omics_name} due to insufficient minority samples ({train_binary_counts_omics.min()}). Requires at least {min_samples_needed}.")
                continue # Skip this technique

            if len(np.unique(y_train_omics_binary)) < 2:
                print(f"  Skipping {technique_name} for {omics_name} due to insufficient classes ({len(np.unique(y_train_omics_binary))}) in training data.")
                continue # Skip this technique


            print(f"  Applying {technique_name} for {omics_name}...")
            X_train_res, y_train_res = sampler.fit_resample(X_train_omics_scaled, y_train_omics_binary)

            # Train the model on resampled data
            current_model = model_instance_original.__class__(**model_instance_original.get_params())
            # For XGBoost/LightGBM, set scale_pos_weight to 1.0 when sampling is used
            if model_name in ["XGBClassifier", "LGBMClassifier"]:
                 current_model.set_params(scale_pos_weight=1.0)
            # Ensure class_weight is not set if sampling is used (except for NuSVC which is special)
            if model_name in ["LogisticRegression", "DecisionTreeClassifier", "SVC", "RandomForestClassifier", "ExtraTreesClassifier"] and 'class_weight' in current_model.get_params():
                 current_model.set_params(class_weight=None)


            print(f"  Training {model_name} for {omics_name} on resampled data...")
            current_model.fit(X_train_res, y_train_res)


        # --- Store Trained Model and Get Test Probabilities ---
        if current_model is not None:
            print(f"  Getting test probabilities for {model_name} ({omics_name})...")
            # Ensure the model supports predict_proba
            if hasattr(current_model, 'predict_proba'):
                 try:
                     # Get probability of the positive class (1)
                     y_prob_test = current_model.predict_proba(X_test_omics_scaled)[:, 1]
                     test_probabilities[omics_name] = y_prob_test
                     trained_base_models[omics_name] = current_model # Store the trained model
                 except Exception as e:
                     print(f"  Error getting predict_proba for {model_name} ({omics_name}): {e}. Skipping for ensemble.")
            else:
                 print(f"  Model {model_name} ({omics_name}) does not support predict_proba. Skipping for ensemble.")


            # --- Get Feature Importance for Base Model ---
            print(f"  Getting feature importance for {model_name} ({omics_name})...")
            feature_importance_data = get_feature_importance(current_model, feature_names_omics, top_n=50)

            # Store feature importance
            if feature_importance_data: # Only store if importance was extracted
                for feature, importance in feature_importance_data.items():
                    importance_entry = {
                        'Omics': omics_name,
                        'Model': model_name,
                        'Technique': technique_name,
                        'Feature': feature,
                        'Importance': importance,
                    }
                    all_feature_importances.append(importance_entry)
            else:
                 # Store a placeholder if feature importance is not available
                 importance_entry = {
                        'Omics': omics_name,
                        'Model': model_name,
                        'Technique': technique_name,
                        'Feature': 'N/A',
                        'Importance': np.nan,
                    }
                 all_feature_importances.append(importance_entry)


    except Exception as e:
        print(f"Error processing {model_name} with {technique_name} for {omics_name}: {e}")
        # Note: We don't add metrics here, only for the final ensemble.
        # We also don't add test probabilities for failed models.
        # Add a placeholder for feature importance if an error occurred
        importance_entry = {
            'Omics': omics_name,
            'Model': model_name,
            'Technique': f"{technique_name} (Failed)",
            'Feature': 'Error',
            'Importance': np.nan,
        }
        all_feature_importances.append(importance_entry)


# --- Ensemble Prediction and Evaluation ---
print("\n--- Building and Evaluating Ensemble Model ---")

# Check if we have trained models with probabilities to form an ensemble
if not test_probabilities:
    print("No base models successfully trained with probability prediction. Cannot build ensemble. Exiting.")
    # Print collected feature importances even if ensemble failed
    if all_feature_importances:
        summary_importance_df = pd.DataFrame(all_feature_importances)
        print("\nIndividual Base Model Feature Importances (Ensemble Failed):")
        print(summary_importance_df.to_string())
    else:
        print("\nNo feature importances were collected.")
    sys.exit()

# Combine probabilities from base models (Soft Voting)
# Ensure all probability arrays have the same number of samples as the test set
valid_probs = [probs for probs in test_probabilities.values() if len(probs) == len(y_test_binary)]

if not valid_probs:
    print("No valid test probabilities collected from base models. Cannot build ensemble. Exiting.")
    # Print collected feature importances even if ensemble failed
    if all_feature_importances:
        summary_importance_df = pd.DataFrame(all_feature_importances)
        print("\nIndividual Base Model Feature Importances (Ensemble Failed):")
        print(summary_importance_df.to_string())
    else:
        print("\nNo feature importances were collected.")
    sys.exit()


# Average the probabilities
ensemble_probabilities = np.mean(valid_probs, axis=0)

# Make final predictions (threshold at 0.5)
ensemble_predictions = (ensemble_probabilities >= 0.5).astype(int)

# Evaluate the ensemble model
print("\nEvaluating Ensemble Model...")
ensemble_metrics = evaluate_binary_model(
    y_test_binary,
    ensemble_predictions,
    y_prob_binary=ensemble_probabilities,
    model_name="Ensemble (Soft Voting)",
    omics_name="All Omics",
    technique_name="Combined"
)

# Store ensemble metrics
all_metrics = [] # Reset metrics list to only store ensemble metrics
metric_entry = {
    'Omics': 'All Omics (Ensemble)',
    'Model': 'Ensemble (Soft Voting)',
    'Technique': 'Combined',
}
metric_entry.update(ensemble_metrics)
all_metrics.append(metric_entry)


print("\n--- Ensemble Evaluation Complete ---")

# --- Print Summary Tables ---

print("\n--- Ensemble Metrics ---")
if all_metrics:
    summary_metrics_df = pd.DataFrame(all_metrics)
    # Reorder columns for better readability
    metric_cols = ["Accuracy", "Balanced Accuracy", "Precision", "Recall (Sensitivity)",
                   "Specificity", "F1-Score", "G-Mean", "ROC-AUC", "PR-AUC", "MCC"]
    summary_cols = ["Omics", "Model", "Technique"] + metric_cols
    summary_metrics_df = summary_metrics_df[summary_cols]

    # Format numerical columns for display
    for col in metric_cols:
         if col in summary_metrics_df.columns:
             summary_metrics_df[col] = summary_metrics_df[col].apply(lambda x: '{:.4f}'.format(x) if isinstance(x, (int, float)) and not np.isnan(x) else str(x))

    # Print the metrics table
    print(summary_metrics_df.to_string())
else:
    print("No ensemble metrics were generated.")

print("\n--- Ensemble Confusion Matrix ---")
if len(np.unique(y_test_binary)) > 1:
    cm = confusion_matrix(y_test_binary, ensemble_predictions)
    cm_df = pd.DataFrame(cm, index=['Actual 0 (Not Early)', 'Actual 1 (Early)'], columns=['Predicted 0 (Not Early)', 'Predicted 1 (Early)'])
    print(cm_df)
else:
    print("Cannot generate confusion matrix: Test set contains fewer than 2 classes.")


print("\n--- Individual Base Model Feature Importances (Top 50) ---")
if all_feature_importances:
    summary_importance_df = pd.DataFrame(all_feature_importances)
    # Print the feature importance table
    print(summary_importance_df.to_string())
else:
    print("No feature importances were extracted from base models.")

print("\nScript finished.")


Data loaded successfully.

Filtering omics data based on pooled features...
Dropping header row from CNV data.
Data filtering complete.
RNA filtered shape: (642, 458)
MiRNA filtered shape: (631, 359)
CNV filtered shape: (624, 481)
Methylation filtered shape: (627, 478)
Protein filtered shape: (624, 314)

--- Processing Protein Data ---
Merging Protein data...
Merge complete. Shape: (624, 315)

--- Processing Methylation Data ---
Merging Methylation data...
Merge complete. Shape: (627, 479)

--- Processing MiRNA Data ---
Merging MiRNA data...
Merge complete. Shape: (631, 360)

--- Processing CNV Data ---
Merging CNV data...
Merge complete. Shape: (624, 482)

--- Processing RNA Data ---
Merging RNA data...
Merge complete. Shape: (631, 459)

Found 624 common samples across all omics.

Data split into 468 training samples and 156 testing samples.
Binary Test set class distribution: binary_target
0     49
1    107
Name: count, dtype: int64

Training individual base models and collecting tes

c:\Users\BITS\.basilisk\1.18.0\0\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\BITS\.basilisk\1.18.0\0\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\BITS\.basilisk\1.18.0\0\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\BITS\.basilisk\1.18.0\0\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\BITS\.basilisk\1.18.0\0\lib\subprocess.py", line 1456, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


  Getting test probabilities for ExtraTreesClassifier (Protein)...
  Getting feature importance for ExtraTreesClassifier (Protein)...

--- Training Methylation Base Model ---
Missing values handled for Methylation using median imputation.
  Applying ADASYN for Methylation...
  Training RandomForestClassifier for Methylation on resampled data...
  Getting test probabilities for RandomForestClassifier (Methylation)...
  Getting feature importance for RandomForestClassifier (Methylation)...

--- Training MiRNA Base Model ---
Missing values handled for MiRNA using median imputation.
  Applying ADASYN for MiRNA...
  Training RandomForestClassifier for MiRNA on resampled data...
  Getting test probabilities for RandomForestClassifier (MiRNA)...
  Getting feature importance for RandomForestClassifier (MiRNA)...

--- Training CNV Base Model ---
Missing values handled for CNV using median imputation.
  Training RandomForestClassifier for CNV with Class Weighting (Balanced)...
  Getting test pro

In [5]:
import pandas as pd 

In [ ]:
print(pooled_data.head()) # Display the first few rows of the pooled data for verification

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score,
    matthews_corrcoef, precision_score, recall_score,
    average_precision_score, confusion_matrix, roc_curve, auc
)
import numpy as np
import re
import sys
import warnings
import os
import matplotlib.pyplot as plt
from scipy.stats import ranksums

# Import imblearn techniques
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek, SMOTEENN

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# --- Data Loading and Filtering ---
try:
    rna_data = pd.read_csv('D:\\extensive analysis data\\rna_samples_with_gene_names.csv')
    mirna_data = pd.read_csv('D:\\extensive analysis data\\miRNA_samples.csv')
    methylation_data = pd.read_csv("D:\\extensive analysis data\\methylation_samples.csv")
    cnv_data = pd.read_csv("D:\\extensive analysis data\\cnv_samples.csv")
    protein_data = pd.read_csv("D:\\extensive analysis data\\protein_samples.csv")
    clin_data = pd.read_csv('D:\\extensive analysis data\\clin_common.csv')
    pooled_data = pd.read_csv('D:\\Downloads\\Top Features - All Pooled.csv')
    print("Data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading data file: {e}")
    sys.exit(1)

# --- Data Filtering ---
print("\nFiltering omics data based on pooled features...")
rna_features = pooled_data['Gene'].dropna().unique().tolist()
rna_data_filtered = rna_data[['Unnamed: 0'] + [col for col in rna_features if col in rna_data.columns]].copy()

mirna_features = pooled_data['miRNA'].dropna().unique().tolist()
mirna_data_filtered = mirna_data[['Unnamed: 0'] + [col for col in mirna_features if col in mirna_data.columns]].copy()

cnv_features = pooled_data['CNV'].dropna().unique().tolist()
if 'gene_id' in cnv_data.iloc[0].astype(str).values:
    cnv_data = cnv_data.iloc[1:].copy()
for col in cnv_data.columns:
    if col != 'Unnamed: 0':
        cnv_data[col] = pd.to_numeric(cnv_data[col], errors='coerce')
cnv_data_filtered = cnv_data[['Unnamed: 0'] + [col for col in cnv_features if col in cnv_data.columns]].copy()

methylation_features = pooled_data['methylation'].dropna().unique().tolist()
methylation_data_filtered = methylation_data[['Unnamed: 0'] + [col for col in methylation_features if col in methylation_data.columns]].copy()

protein_features = pooled_data['Protein'].dropna().unique().tolist()
protein_data_filtered = protein_data[['Unnamed: 0'] + [col for col in protein_features if col in protein_data.columns]].copy()

print("Data filtering complete.")

# --- Helper Functions ---

def clean_sample_ids(sample_id_list):
    cleaned_ids = []
    for sid in sample_id_list:
        sid_str = str(sid)
        match = re.match(r'TCGA-\w{2}-\w{4}-\w{2}', sid_str)
        if match:
            cleaned_ids.append(match.group(0))
        else:
            cleaned_ids.append(sid_str)
    return cleaned_ids

def merge_omics_clinical(omics_df, clinical_df, omics_name):
    omics_df['cleaned_sample_id'] = clean_sample_ids(omics_df['Unnamed: 0'])
    clinical_df['cleaned_sample_id'] = clean_sample_ids(clinical_df['sample_id.1'])
    merged_df = pd.merge(omics_df, clinical_df[['cleaned_sample_id', 'stage_classification']],
                         on='cleaned_sample_id', how='inner')
    return merged_df.drop(['Unnamed: 0'], axis=1)

def handle_missing_values(df, strategy='median'):
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    if strategy == 'median':
        for col in numerical_cols:
            df[col] = df[col].fillna(df[col].median())
    return df

def evaluate_binary_model(y_true_binary, y_pred_binary, y_prob_binary=None):
    metrics = {}
    metrics["Accuracy"] = accuracy_score(y_true_binary, y_pred_binary)
    metrics["Balanced Accuracy"] = balanced_accuracy_score(y_true_binary, y_pred_binary)
    metrics["MCC"] = matthews_corrcoef(y_true_binary, y_pred_binary)
    metrics["Precision"] = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    metrics["Recall (Sensitivity)"] = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    metrics["F1-Score"] = f1_score(y_true_binary, y_pred_binary, zero_division=0)
    if len(np.unique(y_true_binary)) > 1:
        cm = confusion_matrix(y_true_binary, y_pred_binary)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        metrics["Specificity"] = specificity
        metrics["G-Mean"] = np.sqrt(metrics["Recall (Sensitivity)"] * specificity)
        if y_prob_binary is not None:
            metrics["ROC-AUC"] = roc_auc_score(y_true_binary, y_prob_binary)
            metrics["PR-AUC"] = average_precision_score(y_true_binary, y_prob_binary)
    return metrics

def get_feature_importance(model, feature_names):
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_[0])
    else:
        return {}
    sorted_indices = np.argsort(importances)[::-1]
    return {feature_names[i]: importances[i] for i in sorted_indices[:50]}
    
# =======================================================================
# === START OF NEWLY ADDED SECTION FOR INDIVIDUAL FEATURE ANALYSIS ===
# =======================================================================
def analyze_individual_feature_potential(merged_data_dict):
    """
    Calculates and plots the univariate ROC-AUC for every feature to find its
    individual diagnostic potential.
    """
    print("\n--- Analyzing Individual Feature Diagnostic Potential ---")
    
    univariate_results = []
    output_dir = "univariate_roc_plots"
    os.makedirs(output_dir, exist_ok=True)
    
    for omics_name, merged_df in merged_data_dict.items():
        print(f"  Calculating ROC curves for all {omics_name} features...")
        
        y_true = merged_df['binary_target']
        X_features = merged_df.drop(columns=['cleaned_sample_id', 'stage_classification', 'binary_target'])

        plt.figure(figsize=(12, 10))
        
        for feature_name in X_features.columns:
            feature_values = X_features[feature_name].dropna()
            labels_for_feature = y_true[feature_values.index]

            if len(labels_for_feature.unique()) < 2:
                continue

            fpr, tpr, _ = roc_curve(labels_for_feature, feature_values)
            roc_auc = auc(fpr, tpr)
            
            group_early = feature_values[labels_for_feature == 1]
            group_not_early = feature_values[labels_for_feature == 0]
            p_value = ranksums(group_early, group_not_early).pvalue if len(group_early) > 0 and len(group_not_early) > 0 else np.nan

            univariate_results.append({
                'Omics': omics_name,
                'Feature': feature_name,
                'AUC': roc_auc,
                'P-Value': p_value
            })

            # Add this feature's curve to the combined plot
            plt.plot(fpr, tpr, lw=1, alpha=0.7) # No label to avoid clutter
        
        # Finalize and save the combined plot for the current omics type
        plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Individual ROC Curves for All {omics_name} Features', fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{omics_name}_all_features_roc_curves.png"))
        plt.close() # Close the plot to free memory
        print(f"  Saved combined ROC plot for {omics_name} features.")


    # --- Create and Print Summary Table ---
    if not univariate_results:
        print("No features were analyzed. Cannot generate summary.")
        return

    summary_df = pd.DataFrame(univariate_results).sort_values('AUC', ascending=False).reset_index(drop=True)
    print("\n--- Top 20 Features by Individual Diagnostic Potential (AUC) ---")
    print(summary_df.head(20).to_string())
    summary_df.to_csv("individual_feature_diagnostic_potential.csv", index=False)
    print("\nFull results saved to 'individual_feature_diagnostic_potential.csv'")
# =======================================================================
# === END OF NEWLY ADDED SECTION ===
# =======================================================================

# --- Main Logic ---

# Ensemble Configuration
ensemble_config = {
    "Protein": {"model": ExtraTreesClassifier(random_state=42), "technique": BorderlineSMOTE(random_state=42), "omics_df": protein_data_filtered},
    "Methylation": {"model": RandomForestClassifier(random_state=42), "technique": ADASYN(random_state=42), "omics_df": methylation_data_filtered},
    "MiRNA": {"model": RandomForestClassifier(random_state=42), "technique": ADASYN(random_state=42), "omics_df": mirna_data_filtered},
    "CNV": {"model": RandomForestClassifier(random_state=42), "technique": "Class Weighting (Balanced)", "omics_df": cnv_data_filtered},
    "RNA": {"model": RandomForestClassifier(random_state=42), "technique": BorderlineSMOTE(random_state=42), "omics_df": rna_data_filtered},
}

# Merge all omics data with clinical data
merged_omics_data = {}
for omics_name, config in ensemble_config.items():
    merged_df = merge_omics_clinical(config["omics_df"].copy(), clin_data.copy(), omics_name)
    if merged_df is not None:
        merged_df['binary_target'] = (merged_df['stage_classification'] == 'Early Stage').astype(int)
        merged_omics_data[omics_name] = merged_df

if not merged_omics_data:
    sys.exit("No omics data successfully merged. Cannot proceed.")

# =======================================================================
# ===> THE NEW FUNCTION IS CALLED HERE <===
# =======================================================================
analyze_individual_feature_potential(merged_omics_data)
# =======================================================================

# Find common samples for consistent train/test split
common_sample_ids = set(merged_omics_data[list(merged_omics_data.keys())[0]]['cleaned_sample_id'])
for omics_name in list(merged_omics_data.keys())[1:]:
    common_sample_ids.intersection_update(merged_omics_data[omics_name]['cleaned_sample_id'])
common_sample_ids = list(common_sample_ids)

print(f"\nFound {len(common_sample_ids)} common samples across all omics for model training.")

# Create the final indexed data dictionary for the ensemble
final_indexed_data = {name: df.set_index('cleaned_sample_id') for name, df in merged_omics_data.items()}

# Split common sample IDs for training and testing
train_indices, test_indices = train_test_split(
    common_sample_ids,
    test_size=0.25,
    random_state=42,
    stratify=final_indexed_data['RNA'].loc[common_sample_ids, 'binary_target']
)

y_test_binary = final_indexed_data['RNA'].loc[test_indices, 'binary_target']
print(f"\nData split into {len(train_indices)} training and {len(test_indices)} testing samples.")

# --- Train Base Models and Evaluate Ensemble ---
trained_base_models = {}
all_feature_importances = []
test_probabilities = {}
all_individual_metrics = []

for omics_name, config in ensemble_config.items():
    print(f"\n--- Processing {omics_name} Base Model ---")
    df_indexed = final_indexed_data[omics_name]
    X = df_indexed.drop(['stage_classification', 'binary_target'], axis=1)
    y = df_indexed['binary_target']

    X_train, X_test = X.loc[train_indices], X.loc[test_indices]
    y_train = y.loc[train_indices]

    X_train = handle_missing_values(X_train, strategy='median')
    X_test = handle_missing_values(X_test, strategy='median')

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    feature_names = X_train.columns.tolist()

    model = config["model"]
    technique = config["technique"]
    model_name = type(model).__name__
    technique_name = technique if isinstance(technique, str) else type(technique).__name__

    try:
        if technique_name == "Class Weighting (Balanced)":
            model.set_params(class_weight='balanced')
            model.fit(X_train_scaled, y_train)
        else:
            X_res, y_res = technique.fit_resample(X_train_scaled, y_train)
            model.fit(X_res, y_res)

        # Evaluate and store individual model performance
        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
        
        metrics = evaluate_binary_model(y_test_binary, y_pred, y_prob)
        metric_entry = {'Omics': omics_name, 'Model': model_name, 'Technique': technique_name, **metrics}
        all_individual_metrics.append(metric_entry)

        if y_prob is not None:
            test_probabilities[omics_name] = y_prob
        
        importances = get_feature_importance(model, feature_names)
        for feature, importance in importances.items():
            all_feature_importances.append({'Omics': omics_name, 'Feature': feature, 'Importance': importance})

    except Exception as e:
        print(f"Error training {omics_name} model: {e}")

# --- Ensemble Evaluation ---
if test_probabilities:
    print("\n--- Evaluating Ensemble Model ---")
    ensemble_probs = np.mean([p for p in test_probabilities.values()], axis=0)
    ensemble_preds = (ensemble_probs >= 0.5).astype(int)
    ensemble_metrics = evaluate_binary_model(y_test_binary, ensemble_preds, ensemble_probs)
    
    print("\n--- Ensemble Metrics ---")
    print(pd.DataFrame([ensemble_metrics]).to_string())

    print("\n--- Ensemble Confusion Matrix ---")
    cm = confusion_matrix(y_test_binary, ensemble_preds)
    print(pd.DataFrame(cm, index=['Actual Not Early', 'Actual Early'], columns=['Pred Not Early', 'Pred Early']))
else:
    print("\nNo base models were successfully trained to form an ensemble.")
    
# --- Final Summary Tables ---
print("\n--- Individual Omics Model Performance ---")
if all_individual_metrics:
    print(pd.DataFrame(all_individual_metrics).to_string())

print("\n--- Top 50 Feature Importances from Base Models ---")
if all_feature_importances:
    print(pd.DataFrame(all_feature_importances).sort_values('Importance', ascending=False).head(50).to_string())

print("\nScript finished.")

Data loaded successfully.

Filtering omics data based on pooled features...
Data filtering complete.

--- Analyzing Individual Feature Diagnostic Potential ---
  Calculating ROC curves for all Protein features...
  Saved combined ROC plot for Protein features.
  Calculating ROC curves for all Methylation features...
  Saved combined ROC plot for Methylation features.
  Calculating ROC curves for all MiRNA features...
  Saved combined ROC plot for MiRNA features.
  Calculating ROC curves for all CNV features...
  Saved combined ROC plot for CNV features.
  Calculating ROC curves for all RNA features...
  Saved combined ROC plot for RNA features.

--- Top 20 Features by Individual Diagnostic Potential (AUC) ---
          Omics          Feature       AUC   P-Value
0   Methylation       cg03782130  0.603793  0.000030
1   Methylation       cg10095954  0.602692  0.000037
2   Methylation       cg25663764  0.602325  0.000039
3   Methylation       cg00968561  0.599188  0.000067
4           RNA 

ValueError: Found input variables with inconsistent numbers of samples: [624, 631]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score,
    matthews_corrcoef, precision_score, recall_score,
    average_precision_score, confusion_matrix, roc_curve, auc
)
import numpy as np
import re
import sys
import warnings
import os
import matplotlib.pyplot as plt
from scipy.stats import ranksums

# Import imblearn techniques
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek, SMOTEENN

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# --- Data Loading and Filtering ---
try:
    rna_data = pd.read_csv('D:\\extensive analysis data\\rna_samples_with_gene_names.csv')
    mirna_data = pd.read_csv('D:\\extensive analysis data\\miRNA_samples.csv')
    methylation_data = pd.read_csv("D:\\extensive analysis data\\methylation_samples.csv")
    cnv_data = pd.read_csv("D:\\extensive analysis data\\cnv_samples.csv")
    protein_data = pd.read_csv("D:\\extensive analysis data\\protein_samples.csv")
    clin_data = pd.read_csv('D:\\extensive analysis data\\clin_common.csv')
    pooled_data = pd.read_csv('D:\\Downloads\\Top Features - All Pooled.csv')
    print("Data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading data file: {e}")
    sys.exit(1)

# --- Data Filtering ---
print("\nFiltering omics data based on pooled features...")
rna_features = pooled_data['Gene'].dropna().unique().tolist()
rna_data_filtered = rna_data[['Unnamed: 0'] + [col for col in rna_features if col in rna_data.columns]].copy()

mirna_features = pooled_data['miRNA'].dropna().unique().tolist()
mirna_data_filtered = mirna_data[['Unnamed: 0'] + [col for col in mirna_features if col in mirna_data.columns]].copy()

cnv_features = pooled_data['CNV'].dropna().unique().tolist()
if 'gene_id' in cnv_data.iloc[0].astype(str).values:
    cnv_data = cnv_data.iloc[1:].copy()
for col in cnv_data.columns:
    if col != 'Unnamed: 0':
        cnv_data[col] = pd.to_numeric(cnv_data[col], errors='coerce')
cnv_data_filtered = cnv_data[['Unnamed: 0'] + [col for col in cnv_features if col in cnv_data.columns]].copy()

methylation_features = pooled_data['methylation'].dropna().unique().tolist()
methylation_data_filtered = methylation_data[['Unnamed: 0'] + [col for col in methylation_features if col in methylation_data.columns]].copy()

protein_features = pooled_data['Protein'].dropna().unique().tolist()
protein_data_filtered = protein_data[['Unnamed: 0'] + [col for col in protein_features if col in protein_data.columns]].copy()

print("Data filtering complete.")

# --- Helper Functions ---

def clean_sample_ids(sample_id_list):
    cleaned_ids = []
    for sid in sample_id_list:
        sid_str = str(sid)
        match = re.match(r'TCGA-\w{2}-\w{4}-\w{2}', sid_str)
        if match:
            cleaned_ids.append(match.group(0))
        else:
            cleaned_ids.append(sid_str)
    return cleaned_ids

def merge_omics_clinical(omics_df, clinical_df, omics_name):
    omics_df['cleaned_sample_id'] = clean_sample_ids(omics_df['Unnamed: 0'])
    clinical_df['cleaned_sample_id'] = clean_sample_ids(clinical_df['sample_id.1'])
    merged_df = pd.merge(omics_df, clinical_df[['cleaned_sample_id', 'stage_classification']],
                         on='cleaned_sample_id', how='inner')
    return merged_df.drop(['Unnamed: 0'], axis=1)

def handle_missing_values(df, strategy='median'):
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    if strategy == 'median':
        for col in numerical_cols:
            df[col] = df[col].fillna(df[col].median())
    return df

def evaluate_binary_model(y_true_binary, y_pred_binary, y_prob_binary=None):
    metrics = {}
    metrics["Accuracy"] = accuracy_score(y_true_binary, y_pred_binary)
    metrics["Balanced Accuracy"] = balanced_accuracy_score(y_true_binary, y_pred_binary)
    metrics["MCC"] = matthews_corrcoef(y_true_binary, y_pred_binary)
    metrics["Precision"] = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    metrics["Recall (Sensitivity)"] = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    metrics["F1-Score"] = f1_score(y_true_binary, y_pred_binary, zero_division=0)
    if len(np.unique(y_true_binary)) > 1:
        cm = confusion_matrix(y_true_binary, y_pred_binary)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        metrics["Specificity"] = specificity
        metrics["G-Mean"] = np.sqrt(metrics["Recall (Sensitivity)"] * specificity)
        if y_prob_binary is not None:
            metrics["ROC-AUC"] = roc_auc_score(y_true_binary, y_prob_binary)
            metrics["PR-AUC"] = average_precision_score(y_true_binary, y_prob_binary)
    return metrics

def get_feature_importance(model, feature_names):
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_[0])
    else:
        return {}
    sorted_indices = np.argsort(importances)[::-1]
    return {feature_names[i]: importances[i] for i in sorted_indices[:50]}
    
def analyze_individual_feature_potential(merged_data_dict):
    print("\n--- Analyzing Individual Feature Diagnostic Potential ---")
    univariate_results = []
    output_dir = "univariate_roc_plots"
    os.makedirs(output_dir, exist_ok=True)
    
    for omics_name, merged_df in merged_data_dict.items():
        print(f"  Calculating ROC curves for all {omics_name} features...")
        y_true = merged_df['binary_target']
        X_features = merged_df.drop(columns=['cleaned_sample_id', 'stage_classification', 'binary_target'])
        plt.figure(figsize=(12, 10))
        for feature_name in X_features.columns:
            feature_values = X_features[feature_name].dropna()
            labels_for_feature = y_true[feature_values.index]
            if len(labels_for_feature.unique()) < 2: continue
            fpr, tpr, _ = roc_curve(labels_for_feature, feature_values)
            roc_auc = auc(fpr, tpr)
            group_early = feature_values[labels_for_feature == 1]
            group_not_early = feature_values[labels_for_feature == 0]
            p_value = ranksums(group_early, group_not_early).pvalue if len(group_early) > 0 and len(group_not_early) > 0 else np.nan
            univariate_results.append({'Omics': omics_name, 'Feature': feature_name, 'AUC': roc_auc, 'P-Value': p_value})
            plt.plot(fpr, tpr, lw=1, alpha=0.7)
        
        plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--'); plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
        plt.title(f'Individual ROC Curves for All {omics_name} Features', fontsize=16)
        plt.tight_layout(); plt.savefig(os.path.join(output_dir, f"{omics_name}_all_features_roc_curves.png")); plt.close()
        print(f"  Saved combined ROC plot for {omics_name} features.")

    if not univariate_results:
        print("No features were analyzed. Cannot generate summary.")
        return
    summary_df = pd.DataFrame(univariate_results).sort_values('AUC', ascending=False).reset_index(drop=True)
    print("\n--- Top 20 Features by Individual Diagnostic Potential (AUC) ---")
    print(summary_df.head(20).to_string())
    summary_df.to_csv("individual_feature_diagnostic_potential.csv", index=False)
    print("\nFull results saved to 'individual_feature_diagnostic_potential.csv'")

# --- Main Logic ---

ensemble_config = {
    "Protein": {"model": ExtraTreesClassifier(random_state=42), "technique": BorderlineSMOTE(random_state=42), "omics_df": protein_data_filtered},
    "Methylation": {"model": RandomForestClassifier(random_state=42), "technique": ADASYN(random_state=42), "omics_df": methylation_data_filtered},
    "MiRNA": {"model": RandomForestClassifier(random_state=42), "technique": ADASYN(random_state=42), "omics_df": mirna_data_filtered},
    "CNV": {"model": RandomForestClassifier(random_state=42), "technique": "Class Weighting (Balanced)", "omics_df": cnv_data_filtered},
    "RNA": {"model": RandomForestClassifier(random_state=42), "technique": BorderlineSMOTE(random_state=42), "omics_df": rna_data_filtered},
}

merged_omics_data = {}
for omics_name, config in ensemble_config.items():
    merged_df = merge_omics_clinical(config["omics_df"].copy(), clin_data.copy(), omics_name)
    if merged_df is not None:
        merged_df['binary_target'] = (merged_df['stage_classification'] == 'Early Stage').astype(int)
        merged_omics_data[omics_name] = merged_df

if not merged_omics_data:
    sys.exit("No omics data successfully merged. Cannot proceed.")

analyze_individual_feature_potential(merged_omics_data)

# =======================================================================
# === START OF FIX FOR ValueError ===
# =======================================================================
print("\n--- Preparing data for ensemble model ---")
# De-duplicate each dataframe to ensure one entry per sample ID
for omics_name, df in merged_omics_data.items():
    if df['cleaned_sample_id'].duplicated().any():
        print(f"  Found and removed duplicate sample IDs in {omics_name} data.")
        merged_omics_data[omics_name] = df.drop_duplicates(subset='cleaned_sample_id', keep='first')

# Now, create the indexed data dictionary
final_indexed_data = {name: df.set_index('cleaned_sample_id') for name, df in merged_omics_data.items()}

# Find common samples AFTER de-duplication
common_sample_ids = set(final_indexed_data[list(final_indexed_data.keys())[0]].index)
for omics_name in list(final_indexed_data.keys())[1:]:
    common_sample_ids.intersection_update(final_indexed_data[omics_name].index)
common_sample_ids = list(common_sample_ids)

print(f"Found {len(common_sample_ids)} common, unique samples across all omics for model training.")
# =======================================================================
# === END OF FIX ===
# =======================================================================

# Split common sample IDs for training and testing
train_indices, test_indices = train_test_split(
    common_sample_ids,
    test_size=0.25,
    random_state=42,
    stratify=final_indexed_data['RNA'].loc[common_sample_ids, 'binary_target']
)

y_test_binary = final_indexed_data['RNA'].loc[test_indices, 'binary_target']
print(f"\nData split into {len(train_indices)} training and {len(test_indices)} testing samples.")

# --- Train Base Models and Evaluate Ensemble ---
# (The rest of the script remains the same)
trained_base_models = {}
all_feature_importances = []
test_probabilities = {}
all_individual_metrics = []

for omics_name, config in ensemble_config.items():
    print(f"\n--- Processing {omics_name} Base Model ---")
    df_indexed = final_indexed_data[omics_name]
    
    # Filter to only common samples before separating X and y
    df_common = df_indexed.loc[common_sample_ids]
    X = df_common.drop(['stage_classification', 'binary_target'], axis=1)
    y = df_common['binary_target']

    X_train, X_test = X.loc[train_indices], X.loc[test_indices]
    y_train = y.loc[train_indices]

    X_train = handle_missing_values(X_train, strategy='median')
    X_test = handle_missing_values(X_test, strategy='median')

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    feature_names = X_train.columns.tolist()

    model = config["model"]
    technique = config["technique"]
    model_name = type(model).__name__
    technique_name = technique if isinstance(technique, str) else type(technique).__name__

    try:
        if technique_name == "Class Weighting (Balanced)":
            model.set_params(class_weight='balanced')
            model.fit(X_train_scaled, y_train)
        else:
            X_res, y_res = technique.fit_resample(X_train_scaled, y_train)
            model.fit(X_res, y_res)

        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
        
        metrics = evaluate_binary_model(y_test_binary, y_pred, y_prob)
        metric_entry = {'Omics': omics_name, 'Model': model_name, 'Technique': technique_name, **metrics}
        all_individual_metrics.append(metric_entry)

        if y_prob is not None:
            test_probabilities[omics_name] = y_prob
        
        importances = get_feature_importance(model, feature_names)
        for feature, importance in importances.items():
            all_feature_importances.append({'Omics': omics_name, 'Feature': feature, 'Importance': importance})

    except Exception as e:
        print(f"Error training {omics_name} model: {e}")

# --- Ensemble Evaluation ---
if test_probabilities:
    print("\n--- Evaluating Ensemble Model ---")
    ensemble_probs = np.mean([p for p in test_probabilities.values()], axis=0)
    ensemble_preds = (ensemble_probs >= 0.5).astype(int)
    ensemble_metrics = evaluate_binary_model(y_test_binary, ensemble_preds, ensemble_probs)
    
    print("\n--- Ensemble Metrics ---")
    print(pd.DataFrame([ensemble_metrics]).to_string())

    print("\n--- Ensemble Confusion Matrix ---")
    cm = confusion_matrix(y_test_binary, ensemble_preds)
    print(pd.DataFrame(cm, index=['Actual Not Early', 'Actual Early'], columns=['Pred Not Early', 'Pred Early']))
else:
    print("\nNo base models were successfully trained to form an ensemble.")
    
# --- Final Summary Tables ---
print("\n--- Individual Omics Model Performance ---")
if all_individual_metrics:
    print(pd.DataFrame(all_individual_metrics).to_string())

print("\n--- Top 50 Feature Importances from Base Models ---")
if all_feature_importances:
    print(pd.DataFrame(all_feature_importances).sort_values('Importance', ascending=False).head(50).to_string())

print("\nScript finished.")